In [48]:
import pandas as pd
import numpy as np
import wfdb
import ast

def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal[:, 0] for signal, meta in data])  # Sélectionne uniquement le premier canal
    return data

path = ''
sampling_rate = 100

# Charger et convertir les données d'annotations
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Charger les données brutes des signaux en ne conservant que le premier canal
X = load_raw_data(Y, sampling_rate, path)

# Charger scp_statements.csv pour l'agrégation diagnostique
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Appliquer la superclasse diagnostique
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Séparer les données en train et test
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass


In [64]:
pd.DataFrame(X_train)

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-0.119,-0.116,-0.120,-0.117,-0.103,-0.097,-0.119,-0.096,-0.048,-0.037,...,0.198,0.194,0.115,0.107,0.107,0.106,0.090,0.069,0.086,0.022
1,0.004,-0.020,-0.053,-0.056,-0.062,-0.065,-0.061,-0.061,-0.064,-0.061,...,-0.035,-0.045,0.004,0.044,0.507,0.554,0.316,0.121,-0.326,-0.348
2,-0.029,-0.035,-0.054,-0.078,-0.088,-0.022,0.346,0.784,0.426,-0.076,...,-0.040,-0.051,-0.026,-0.032,-0.052,-0.039,-0.034,-0.029,-0.048,-0.049
3,-0.054,-0.053,-0.063,-0.060,-0.050,-0.054,-0.059,-0.058,-0.054,-0.051,...,-0.312,-0.511,-0.280,-0.076,-0.012,0.001,-0.003,0.026,0.026,0.028
4,-0.034,-0.038,-0.057,-0.066,-0.080,-0.085,-0.058,-0.061,-0.068,-0.065,...,0.005,0.001,0.003,0.013,0.018,-0.001,0.007,0.000,-0.003,-0.012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19629,-0.052,-0.077,-0.042,0.008,-0.044,0.035,0.122,0.131,0.129,0.073,...,-0.143,-0.125,-0.104,-0.097,-0.052,0.010,0.019,-0.024,-0.061,-0.050
19630,-0.050,-0.032,0.008,0.014,0.010,0.032,0.005,-0.048,-0.041,-0.038,...,-0.023,-0.052,-0.062,-0.059,-0.074,-0.069,-0.107,0.058,0.501,0.584
19631,0.038,0.078,-0.014,0.072,0.167,0.071,0.108,0.043,0.001,0.031,...,-0.081,0.012,0.504,0.613,0.328,0.061,0.037,-0.106,-0.045,0.463
19632,-0.057,-0.041,0.003,0.000,0.022,0.038,0.019,0.034,0.045,0.017,...,0.074,0.112,0.116,0.115,0.112,0.096,0.051,0.033,0.027,-0.006


In [59]:
# Convertir chaque liste d'étiquettes en série
y_train_series = y_train.apply(pd.Series).stack()
y_test_series = y_test.apply(pd.Series).stack()

# Extraire les éléments uniques de chaque série
unique_labels_train = y_train_series.unique()
unique_labels_test = y_test_series.unique()

# Convertir les éléments uniques en une liste ou un tableau numpy
unique_labels = np.union1d(unique_labels_train, unique_labels_test)

# Maintenant, utilisez l'encodeur approprié
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(unique_labels)

# Transformez les étiquettes en valeurs numériques
y_train_encoded = label_encoder.transform(y_train_series)
y_test_encoded = label_encoder.transform(y_test_series)

# Assurez-vous de remodeler les étiquettes après la transformation
y_train_encoded = y_train_encoded.reshape(-1, 1)
y_test_encoded = y_test_encoded.reshape(-1, 1)


In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Créer un classificateur d'arbre de décision
tree_classifier = DecisionTreeClassifier()

# Entraîner le modèle sur les données d'entraînement
tree_classifier.fit(X_train, y_train_encoded)

# Prédire sur les données de test
y_pred_encoded = tree_classifier.predict(X_test)

# Convertir les prédictions en étiquettes originales
y_pred_labels = label_encoder.inverse_transform(y_pred_encoded)

# Afficher le rapport de classification
print(classification_report(y_test_series, y_pred_labels))


ValueError: Number of labels=25025 does not match number of samples=19634

In [62]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train_encoded:", y_train_encoded.shape)


Shape of X_train: (19634, 1000)
Shape of y_train_encoded: (25025, 1)


In [53]:
from sklearn.metrics import classification_report

# Convertir les prédictions en labels
y_pred_labels = mlb.inverse_transform(y_pred)

# Convertir les vraies étiquettes en une matrice binaire
y_test_encoded = mlb.transform(y_test)

# Afficher le rapport de classification
print(classification_report(y_test_encoded, y_pred_labels))


ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.